In [1]:
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import scipy.io
import seaborn as sns
import warnings
from sklearn.cluster import KMeans

In [6]:
graph = nx.Graph()
with open('./Data/Wiki-Vote.txt' , "r") as file:
    f = file.readlines()
    data = []
    for line in f:
        data = line.strip().split()
        graph.add_edge(data[0], data[1])

In [9]:
m = graph.number_of_edges()
num_nodes = graph.number_of_nodes()
print("Number of nodes: ", num_nodes)
print("Number of edges: ", m)

Number of nodes:  7120
Number of edges:  100766


In [25]:
def modularity_mat(graph):
    m = graph.number_of_edges()
    num_nodes = graph.number_of_nodes()
    nodes = graph.nodes()
    A = np.zeros((num_nodes, num_nodes))
    for i,node1  in enumerate(nodes):
        for j,node2 in enumerate(nodes):
            b = graph.degree(node1)*graph.degree(node2)/(2*m)
            if graph.has_edge(node1, node2):
                a = 1
            else:
                a = 0
            A[i,j] = a-b
    return A

def dominant_eigenvector(A):
    eigenvalues, eigenvectors = np.linalg.eig(A)
    idx = np.argsort(eigenvalues)
    eigenvalues = eigenvalues[idx]
    eigenvectors = eigenvectors[:,idx]
    return eigenvectors[:,-1]


In [12]:
modularity_matrix = modularity_mat(graph)
print("Modularity matrix: ", modularity_matrix)

Modularity matrix:  [[-7.93918584e-05  9.99980152e-01  9.99980152e-01 ... -1.98479646e-05
  -3.96959292e-05 -1.98479646e-05]
 [ 9.99980152e-01 -4.96199115e-06 -4.96199115e-06 ... -4.96199115e-06
  -9.92398230e-06 -4.96199115e-06]
 [ 9.99980152e-01 -4.96199115e-06 -4.96199115e-06 ... -4.96199115e-06
  -9.92398230e-06 -4.96199115e-06]
 ...
 [-1.98479646e-05 -4.96199115e-06 -4.96199115e-06 ... -4.96199115e-06
  -9.92398230e-06 -4.96199115e-06]
 [-3.96959292e-05 -9.92398230e-06 -9.92398230e-06 ... -9.92398230e-06
  -1.98479646e-05 -9.92398230e-06]
 [-1.98479646e-05 -4.96199115e-06 -4.96199115e-06 ... -4.96199115e-06
  -9.92398230e-06 -4.96199115e-06]]


In [17]:
dominant_eigenvector = dominant_eigenvector(modularity_matrix)
print("Dominant eigenvector: ", dominant_eigenvector)



Dominant eigenvector:  [-8.17989159e-06+0.j -2.11486281e-06+0.j -2.11486281e-06+0.j ...
  2.05998311e-04+0.j  3.79763926e-05+0.j  2.27060906e-05+0.j]


In [29]:
def binary_clustering(graph, Y):
    partition1= np.where(Y >= 0)[0]
    partition2= np.where(Y < 0)[0]
    print("Number of nodes in cluster 1: ", len(partition1))
    print("Number of nodes in cluster 2: ", len(partition2))
    nodes = list(graph.nodes())
    cluster1 = [nodes[i] for i in partition1]
    cluster2 = [nodes[i] for i in partition2]
    list_of_clusters = [cluster1, cluster2]
    return list_of_clusters, cluster1, cluster2
    

In [36]:
def modularity_fun(clusters,G):
    m = G.number_of_edges()
    sums = []
    for cluster in clusters:
        sum = 0
        
        sub_graph = G.subgraph(cluster)
        nodes = list(sub_graph.nodes())
        for node1 in nodes:
            for node2 in nodes:
                t = sub_graph.degree[node1] * sub_graph.degree[node2]/(2*m)
                if sub_graph.has_edge(node1,node2):
                    a = 1
                else:
                    a = 0
                sum+= a - t
            sums.append(sum)
    return(np.sum(sums)/(2*m))

In [37]:
total_clusters =[]
total_modularity=[]
def clustering(graph, modularity,list_of_clusters):
    
    index = list_of_clusters.index(set(graph.nodes()))
    last_cluster = list_of_clusters.pop(index)
    print('----------------------------------------------------------------------------')
    print("\nlength of last cluster: ", len(last_cluster))

    M = modularity_mat(graph)
    D = dominant_eigenvector(M)
    print("dominant eigenvector: ", D)
    res, sub1, sub2 = binary_clustering(graph, D)

    list_of_clusters.append(set(res[0]))
    list_of_clusters.append(set(res[1]))

    print("length of cluster 1: ", len(set(res[0])))
    print("length of cluster 2: ", len(set(res[1])))

    total_clusters.append(list_of_clusters)

    modularity_new = modularity_fun(list_of_clusters, graph)
    total_modularity.append(modularity_new)
    print("new modularity: ", modularity_new)

    if modularity_new < modularity:
        list_of_clusters.pop()
        list_of_clusters.pop()
        list_of_clusters.append(last_cluster)
        return list_of_clusters


    clustering(sub1, modularity_new, list_of_clusters)
    clustering(sub2, modularity_new, list_of_clusters)
    return list_of_clusters

In [38]:
modularity =0
list_of_clusters = []
list_of_clusters.append(set(graph.nodes()))
clusters = clustering(graph, modularity, list_of_clusters)

----------------------------------------------------------------------------

length of last cluster:  7120
dominant eigenvector:  [-8.17989159e-06+0.j -2.11486281e-06+0.j -2.11486281e-06+0.j ...
  2.05998311e-04+0.j  3.79763926e-05+0.j  2.27060906e-05+0.j]
Number of nodes in cluster 1:  3347
Number of nodes in cluster 2:  3773
length of cluster 1:  3347
length of cluster 2:  3773


KeyboardInterrupt: 